In [1]:
from databases import BDS, PosicaoDm1, Bawm,Crm,BaseExtrato,off_Reliance
from funcoes_datas import FuncoesDatas
from dateutil.relativedelta import relativedelta
import pandas as pd
from datetime import date, timedelta,datetime
from dateutil.relativedelta import relativedelta
import datetime
import numpy as np
import math
from emailer import Email, EmailLer
from pretty_html_table import build_table
from funcoes import CalculosRiscoPort

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
#Chama as classes do databases
crm = Crm()
bawm = Bawm()
datas = FuncoesDatas()
extrato = BaseExtrato()
bds = BDS()
fdt = FuncoesDatas()
calc = CalculosRiscoPort(offshore = True)
rel_off = off_Reliance('Primeny')

#Puxando a ultima data do mes anterior para calculo do extrato
data = datetime.datetime.date(fdt.hoje() - relativedelta(days=fdt.hoje().day -1)+timedelta(days=-1))

In [3]:
#Informações necessárias para otimização dos portfólios


#Traz os de_paras de mandatos (mandato anterior para o atual)
mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_mandato')
mandatos = mandatos[['Anterior','Novo']]
#mandatos = dict(mandatos.values)
    
# #Traz os de_paras de classificação gps para asset Allocation
dicio_classificacoes = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
dicio_classificacoes=dicio_classificacoes[['Classificação GPS','De Para - Asset Allocation']]
dicio_classificacoes = dict(dicio_classificacoes.values)

#Novos mandatos
novos_mandatos = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='Novos_mandatos')
novos_mandatos= novos_mandatos[['Portfolio','Classe','Tático']]

#Benchmarks
benchs = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/20 - Offshore/De Para Ativos.xlsx',sheet_name='de_para_classificao')
benchs=benchs[['De Para - Asset Allocation','Index_Bawm']]

In [4]:
# #Mapeando as informações da base Reliance e linkando com os produtos da base (classificação, isin e ticker do CRM).

#Trazer a última posição disponivel (mes anterior)
carteira_rel = rel_off.carteira_reliance(data)
trazer_data = datetime.datetime.date(carteira_rel['calculado_em'].max())
carteira_rel['calculado_em'] = carteira_rel['calculado_em'].apply(lambda x : datetime.datetime.date(x))
carteira_rel = carteira_rel[carteira_rel['calculado_em'] == trazer_data]
carteira_rel = carteira_rel[~(carteira_rel['Nome_produto'].str.contains('FWD'))&(carteira_rel['SaldoNaData']>=0)]

#Incluir os daldos na composição da carteira
saldos = rel_off.buscar_saldos()


In [ ]:
# #Caso seja necessario buscar um cliente especifico
# carteira_rel['Nome_Cliente'] = carteira_rel['Nome_Cliente'].astype('str')
# #carteira_rel = carteira_rel[carteira_rel['Nome_Cliente'].str.contains('SHIBUYA')]
carteira_rel = carteira_rel[carteira_rel['Nome_Cliente']=='ITABAIANA']
saldos = saldos[saldos['Nome_Cliente']=='ITABAIANA']

In [5]:
#Ajuste da Classe para padrão GPS
classes_padrao = pd.read_excel('classificacao.xlsx')[['ID_Produto','cd_asset_class_name']]
carteira_rel = pd.merge(left = carteira_rel, right=classes_padrao,left_on='cd_security', right_on='ID_Produto', how='left')
carteira_rel['Classe'] = carteira_rel['cd_asset_class_name']
carteira_rel = carteira_rel.drop(columns={'ID_Produto','cd_asset_class_name'})

#Remover todos os valores iguais a 0

carteira_rel = carteira_rel.dropna(subset=['SaldoNaData'])

#Concatenar as posicoes com os saldos

carteira_rel = carteira_rel[['Data_Posicao', 'Nome_Cliente', 'cd_security','Nome_produto','Classe','Subcategoria','quantidade','SaldoNaData']].rename(columns={'Subcategoria':'Subclasse'})
saldos['cd_security'] = 1
saldos = saldos[['Data_Posicao', 'Nome_Cliente','cd_security', 'Nome_produto','Classe','Subclasse','quantidade','SaldoNaData']]

carteira_rel = pd.concat([carteira_rel,saldos]).sort_values(by=['Nome_Cliente'])
carteira_copy = carteira_rel

#Ajustar o nome das classes

carteira_rel['Classe'] = carteira_rel['Classe'].str.lower()
carteira_rel['Nome_Cliente'] = carteira_rel['Nome_Cliente'].astype('str')
carteira_rel['Classe']  = carteira_rel['Classe'] .apply(lambda x: x.replace(' -','').replace('não classificado','outros'))
carteira_rel=carteira_rel.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData'])

In [8]:
#Incluir peso dos produtos e os PLs nas carteiras

percentual_produto=[]
financeiro_total_spe =[]
financeiro_total =[]
percentual_intraclasse = []

for ind,row in carteira_rel.iterrows():   
    soma = carteira_rel[carteira_rel['Nome_Cliente']==row['Nome_Cliente']]['SaldoNaData'].sum()
    iliquidos = carteira_rel[(carteira_rel['Nome_Cliente']==row['Nome_Cliente'])&(carteira_rel['Classe']=='iliq. p. equity')]['SaldoNaData'].sum()
    financeiro_total_spe.append(soma-iliquidos)
    financeiro_total.append(soma)
    percentual= row['SaldoNaData']/(soma-iliquidos)
    percentual_produto.append(percentual)
    financeiro_intraclasse = carteira_rel[(carteira_rel['Classe']==row['Classe'])&(carteira_rel['Nome_Cliente']==row['Nome_Cliente'])]['SaldoNaData'].sum()
    percentual = row['SaldoNaData']/financeiro_intraclasse
    percentual_intraclasse.append(percentual)
    
    
    #Incluir no DataFrame o calculo acima
carteira_rel['percentual_produto']=percentual_produto
carteira_rel['percentual_produto_intraclasse']=percentual_intraclasse
carteira_rel['Financeiro_Total']=financeiro_total
carteira_rel['Financeiro_Total_s/pe']=financeiro_total_spe                                      
carteira_rel = carteira_rel.sort_values(by=['Classe'])
carteira_rel = carteira_rel.fillna(0)
carteira_rel=carteira_rel.drop_duplicates(subset=['Data_Posicao', 'Nome_Cliente', 'Nome_produto', 'quantidade','SaldoNaData'])#
carteira_rel.to_excel('carteira_reliance_v2.xlsx')

<ipython-input-8-659939519adb>:16: RuntimeWarning: invalid value encountered in double_scalars
  percentual = row['SaldoNaData']/financeiro_intraclasse
<ipython-input-8-659939519adb>:13: RuntimeWarning: divide by zero encountered in double_scalars
  percentual= row['SaldoNaData']/(soma-iliquidos)
<ipython-input-8-659939519adb>:13: RuntimeWarning: invalid value encountered in double_scalars
  percentual= row['SaldoNaData']/(soma-iliquidos)
<ipython-input-8-659939519adb>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  carteira_rel['percentual_produto']=percentual_produto
<ipython-input-8-659939519adb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [9]:
# #Deixando apenas as colunas que interessam para o groupby
peso_te = carteira_rel[carteira_rel['Classe']!='iliq. p. equity']
peso_te = peso_te.groupby(['Data_Posicao','Nome_Cliente','Classe','Financeiro_Total','Financeiro_Total_s/pe']).sum().reset_index()
peso_te = peso_te[['Data_Posicao','Nome_Cliente','Classe','percentual_produto']]
peso_te.to_excel('pesos_carteira_rel.xlsx')

In [11]:
#Rodar a formula de TE e retornar o melhor portfolio para o cliente
carteira_rel = carteira_copy
carteira_rel['NomeSupercarteiraCrm'] = carteira_rel['Nome_Cliente']
carteira_rel['Politica'] = 'Pontos de Risco'
carteira_rel['Officer'] = 'Tamara Alves'
carteira_rel['Controller'] = 'Tamara Alves'
carteira_rel['CodigoProduto'] = 'Rel ' + str(carteira_rel['cd_security'])  
carteira_rel = carteira_rel.rename(columns={'Classe':'classe_sistema','asset_class':'subclasse_sistema','moeda':'Moeda'})

KeyError: "['Moeda', 'PU', 'subclasse_sistema', 'Esteira', 'ISIN'] not in index"